In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Apr 22 15:32:58 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
import keras
import os
import glob
import sys
from music21 import converter, instrument, note, chord

Using TensorFlow backend.


In [4]:
#Getting the directory with midi files
folder = os.path.join('/content/drive/My Drive/Midi_dataset') #path
for filename in os.listdir(folder):
  if filename.endswith(".mid"):
    print(filename)

waldstein_1.mid
mozk175a.mid
waldstein_3.mid
mozk545c.mid
mozk545a.mid
beethoven_opus90_2.mid
waldstein_2.mid
beethoven_opus90_1.mid
mozk545b.mid
ty_februar.mid
mozk466a.mid
beethoven_opus10_1.mid
mozk450c.mid
mozk309a.mid
mozk450b.mid
mozk450a.mid
mozk332a.mid
mozk488a.mid
mozk467a.mid
mozk467c.mid
mozk332b.mid
mozk332c.mid
ty_juni.mid
mozk488b.mid
ty_april.mid
mozk467b.mid
mozk466b.mid
mozk333c.mid
mozk488c.mid
beethoven_opus10_2.mid
mozk309b.mid
mozk309c.mid
beethoven_opus10_3.mid
elise.mid
ty_maerz.mid
mozk333b.mid
mozk466c.mid
ty_november.mid
mozk246b.mid
beethoven_opus22_3.mid
beethoven_opus22_4.mid
beethoven_hammerklavier_3.mid
beethoven_hammerklavier_2.mid
beethoven_hammerklavier_1.mid
mozk246c.mid
ty_august.mid
beethoven_hammerklavier_4.mid
mozk311b.mid
ty_mai.mid
beethoven_opus22_2.mid
beethoven_les_adieux_1.mid
ty_oktober.mid
beethoven_opus22_1.mid
mond_2.mid
mozk281c.mid
pathetique_3.mid
mozk311c.mid
beethoven_les_adieux_2.mid
ty_september.mid
mozk331c.mid
mond_1.mid
pathet

In [5]:
#main parsing function, decompose each file in notes
notes = []

os.chdir(folder)
for file in glob.glob("*.mid"):
    midi = converter.parse(file)

    print("Parsing %s" % file)
    
    notes_to_parse = None

    try: # file has instrument parts
        s2 = instrument.partitionByInstrument(midi)
        notes_to_parse = s2.parts[0].recurse() 
    except: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

print(notes)

Parsing waldstein_1.mid
Parsing mozk175a.mid
Parsing waldstein_3.mid
Parsing mozk545c.mid
Parsing mozk545a.mid
Parsing beethoven_opus90_2.mid
Parsing waldstein_2.mid
Parsing beethoven_opus90_1.mid
Parsing mozk545b.mid
Parsing ty_februar.mid
Parsing mozk466a.mid
Parsing beethoven_opus10_1.mid
Parsing mozk450c.mid
Parsing mozk309a.mid
Parsing mozk450b.mid
Parsing mozk450a.mid
Parsing mozk332a.mid
Parsing mozk488a.mid
Parsing mozk467a.mid
Parsing mozk467c.mid
Parsing mozk332b.mid
Parsing mozk332c.mid
Parsing ty_juni.mid
Parsing mozk488b.mid
Parsing ty_april.mid
Parsing mozk467b.mid
Parsing mozk466b.mid
Parsing mozk333c.mid
Parsing mozk488c.mid
Parsing beethoven_opus10_2.mid
Parsing mozk309b.mid
Parsing mozk309c.mid
Parsing beethoven_opus10_3.mid
Parsing elise.mid
Parsing ty_maerz.mid
Parsing mozk333b.mid
Parsing mozk466c.mid
Parsing ty_november.mid
Parsing mozk246b.mid
Parsing beethoven_opus22_3.mid
Parsing beethoven_opus22_4.mid
Parsing beethoven_hammerklavier_3.mid
Parsing beethoven_ham

In [0]:
#save the notes list to an external file
import pickle
outfile = open('data_notes.obj', 'wb')
pickle.dump(notes, outfile)

In [7]:
import numpy as np
from keras.utils import np_utils
#Prepare the sequences used by the Neural Network

sequence_length = 100

# get all pitch names
n_vocab = len(set(notes))
pitchnames = sorted(set(item for item in notes))

# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

network_input = []
network_output = []

# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)

# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)

network_output = np_utils.to_categorical(network_output)

print(network_input)
print(network_output)

[[[0.85057471]
  [0.05172414]
  [0.57471264]
  ...
  [0.51149425]
  [0.51149425]
  [0.51149425]]

 [[0.05172414]
  [0.57471264]
  [0.05172414]
  ...
  [0.51149425]
  [0.51149425]
  [0.51149425]]

 [[0.57471264]
  [0.05172414]
  [0.57471264]
  ...
  [0.51149425]
  [0.51149425]
  [0.82471264]]

 ...

 [[0.87643678]
  [0.8591954 ]
  [0.87643678]
  ...
  [0.40229885]
  [0.02873563]
  [0.80172414]]

 [[0.8591954 ]
  [0.87643678]
  [0.89367816]
  ...
  [0.02873563]
  [0.80172414]
  [0.94827586]]

 [[0.87643678]
  [0.89367816]
  [0.8591954 ]
  ...
  [0.80172414]
  [0.94827586]
  [0.18390805]]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [0]:
model = Sequential()
model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
model.add(LSTM(512))
model.add(BatchNorm())
model.add(Dropout(0.3))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(BatchNorm())
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [0]:
model = Sequential()
model.add(Bidirectional(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]), 
        recurrent_dropout=0.3,
        return_sequences=True
    )))
model.add(LSTM(512))
model.add(BatchNorm())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(BatchNorm())
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [0]:
# Model Checkpoint
filepath = os.path.abspath("weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5")
keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=False, save_weights_only=False, mode='min')
callbacks_list = [checkpoint]

In [23]:
model.fit(network_input, network_output, epochs=50, batch_size=128, callbacks=callbacks_list)

Epoch 1/50
220778/220778 [==============================] - 782s 4ms/step - loss: 4.4021 - accuracy: 0.0410


KeyError: ignored

In [0]:
print(model.summary())